## Define a Class Object

In [5]:
class MyFirstClass:
    favorite_number = 41
    def print_number(number):
        value= 63
        print(number)

***Attribute Resolution on Class Objects***
---
Our `class` object also has `attributes` (the ones we defined in its body!) that we can access with a new-ish syntax - asking Python for `some_object.some_attribute` instructs the name `some_attribute` to be resolved on the `some_object` object. For us, this looks like

In [6]:
print(type(MyFirstClass.__dict__))  # something that acts a lot like a normal dictionary
print('favorite_number' in MyFirstClass.__dict__)  # True
print('greet' in MyFirstClass.__dict__)  # True

<class 'mappingproxy'>
True
False


In [7]:
MyFirstClass.__dict__

mappingproxy({'__module__': '__main__',
              'favorite_number': 41,
              'print_number': <function __main__.MyFirstClass.print_number(number)>,
              '__dict__': <attribute '__dict__' of 'MyFirstClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'MyFirstClass' objects>,
              '__doc__': None})

In [9]:
getattr(MyFirstClass, 'favorite_number')

41

In [18]:
class Dog:
    def __init__(self, name, tricks=None ):
        self.name = name
        if tricks : 
            self.tricks = tricks 
        else: 
            self.tricks = set() 
    
    def teach(self, trick):
        self.tricks.add(trick)
        
# Change the broken code above so that the following lines work:

buddy = Dog('Buddy')
pascal = Dog('Pascal')
kimber = Dog('Kimber', tricks={'lie down', 'shake'})
buddy.teach('sit')
pascal.teach('fetch')
buddy.teach('roll over')
kimber.teach('fetch')
print(buddy.tricks)  # {'roll over', 'sit'}
print(pascal.tricks)  # {'fetch'}
print(kimber.tricks)  # {'fetch', 'shake', 'lie down'}

{'sit', 'roll over'}
{'fetch'}
{'fetch', 'shake', 'lie down'}


In [24]:
# This one's a bit different, representing an unusual (and honestly,
# not recommended) strategy for tracking users that sign up for a service.

class User:
    # An (intentionally shared) collection storing users who sign up for some hypothetical service.
    # There's only one set of members, so it lives at the class level!
    members = set()
    def __init__(self, name , members = set()):
        self.name = name
        self.members = members  # Not signed up to begin with.
    def sign_up(self):
        User.members.add(self.name)

# Change the code above so that the following lines work:
# 
sarah = User('sarah')
heather = User('heather')
cristina = User('cristina')
print(User.members)  # set()
heather.sign_up()
cristina.sign_up()
print(User.members)  # {'heather', 'cristina'}

set()
{'cristina', 'heather'}


In [15]:
id(pascal.tricks)

2403939576736

***There are two other intriguing method decorators built into Python: `classmethod` and `staticmethod`***
---
There are two other intriguing method decorators built into Python: classmethod and staticmethod

The `@classmethod` > decorator changes method call behavior by passing the class object, not the instance object, as the first argument.

The `@staticmethod` >  decorator changes method call behavior by not supplying either the instance object nor the class object as the first argument.

Class methods are a useful technique for representing factory functions - other ways to create instance objects, but attached to the class itself.

Static methods are a useful technique for attaching utility functions to a class.


In [25]:
class Example:
    def a_normal_method(self, a, b):
        print(self, a, b)
    @classmethod
    def a_class_method(cls, a, b):
        print(cls, a, b)
    @staticmethod
    def a_static_method(a, b):
        print(a, b)

ex = Example()

In [30]:
ex.a_normal_method(1,2)

<__main__.Example object at 0x0000022FB53232B0> 1 2


In [31]:
ex.a_class_method(1,2)

<class '__main__.Example'> 1 2


The goal of this exercise was to gain experience using the common (and uncommon) tools available in Python for designing a class - specifically, a customer that's part of a subscription service.

We define a new Customer class object, inside of which are many methods. The __init__ method is responsibly for finishing the initialization of a new Customer, and it adds a first_name, surname, _tier, and _cost attribute to a newly-formed instance object.

Then, the bill_for method can take the instance object and some number of months, and return how much to bill the customer (at the customer's ._cost for the given number of months. The can_access method takes in an instance object and a unit content (perhaps a dictionary with a 'tier' key), and returns whether the content is free or the content's tier matches the customer's tier.

Finally, we define a property name by decorating a method with the @property decorator, which returns the first name and the surname of the customer, and a class method premium that creates a new premium subscriber from a first name and a surname.

There are many valid ways to design a Customer class - if you followed a different path, not to worry! The goal really was to explore design decisions and gain confidence with the mechanics of designing Python classes.

In [34]:
class Customer:
    def __init__(self , first_name , last_name , tier = ('free' , 0)) : 
        self.f_n  = first_name 
        self.l_n = last_name 
        self._tier = tier[0]
        self._cost = tier[1]
    def can_access(self, content): 
        return content['tier'] == 'free' or content['tier'] == self._tier
    
    def bill_for(self , months):
        return months *self._cost 


    @property 
    def name(self): 
        return f"{self.f_n} {self.l_n}"
    

    @classmethod 
    def premium(cls , first_name , last_name , tier = ('premium' , 10)) : 
        return cls(first_name , last_name , tier)

    


In [35]:
marco = Customer('Marco', 'Polo')  # Defaults to the free tier
print(marco.name)  # Marco Polo
print(marco.can_access({'tier': 'free', 'title': '1812 Overture'}))  # True
print(marco.can_access({'tier': 'premium', 'title': 'William Tell Overture'}))  # False

victoria = Customer.premium("Alexandrina", "Victoria")  # Build a customer around the ('premium', 10$/mo) streaming plan.
print(victoria.can_access({'tier': 'free', 'title': '1812 Overture'}))  # True
print(victoria.can_access({'tier': 'premium', 'title': 'William Tell Overture'}))  # True
print(victoria.bill_for(5))  # => 50 (5 months at 10$/mo)
print(victoria.name)  # Alexandrina Victoria

Marco Polo
True
False
True
True
50
Alexandrina Victoria


In [39]:
Customer._tier

AttributeError: type object 'Customer' has no attribute '_tier'

### Magic Methods

*Python provides tools for making our custom classes "act like" other `built-in` types.*

`If we have an instance object obj of a custom class, and Python needs to evaluate len(obj), Python will attempt to call the magic method obj.__len__. If Python needs to evaluate print(obj), it will call the magic method obj.__str__. If Python needs to evaluate obj + other, it will first attempt to call obj.__add__(other). There are magic methods for nearly all built-in Python behaviors, so by implementing the correct ones, we can make our class act like a Sized container, have a human-readable representation, or even act a bit like a numeric type, able to be added to other objects.`

*The __init__ method that we've seen before falls into this category - it let's us define custom classes that hook into Python's initialization procedure!*

In [40]:
class MagicShoppingCart:
    def __init__(self, items):
        self.items = items
    def __len__(self):
        return sum(self.items.values())
    def __str__(self):
        return f"MagicShoppingCart({self.items})"
    def __contains__(self, item):
        return item in self.items

In [66]:
class Point:
    """Implement your Point class in here!"""
    def __init__(self , x = 0 , y = 0):
        self.x = x 
        self.y = y 
    
    def __str__(self):
        point = tuple([self.x , self.y])
        return f"Point {point} "

    @property 
    def point(self):
        return tuple([self.x ,self.y]) 


    def __add__(self , other):
        x ,y  =  other.point
        return  Point(self.x + x , self.y + y )
        

if __name__ == '__main__':
    # This won't work until you finish implementing the Point class.
    origin = Point()
    point = Point(4, 1)
    other_point = Point(3, -3)
    third_point = point + other_point

    print(point)
    print(other_point)
    print(third_point)

Point (4, 1) 
Point (3, -3) 
Point (7, -2) 


## Classes in Python can derive (inherit) from other classes.
---

``` python 
class DerivedClassName(BaseClassName):
    pass

class MultiplyDerived(Base1, Base2, Base3):
    pass
```
---

The class object `DerivedClassName` has one direct superclass - `BaseClassName`. The class object `MultiplyDerived` has an ordered sequence of direct superclasses - Base1, Base2, and Base3. The class definitions we've seen so far (that look like class ClassName: with no parentheses) inherit from the base type object.

Let's look at an example: the `MotorVehicle` (super)class and the `Car` (derived) class.




In [67]:
class MotorVehicle:
    def __init__(self, range):
        self.range = range
        self.tank = range
    def travel(self, distance):
        if distance > self.tank:
            print(f"Not enough in the tank. Only traveled {self.tank} kilometers.")
            self.tank = 0
        else:
            print(f"VOOOM! Traveled {distance} kilometers.")
            self.tank -= distance
    def refuel(self):
        print("Refueling...")
        self.tank = self.range
    def __str__(self):
        return(f"Vehicle(range={self.range}, tank={self.tank})")

class Car(MotorVehicle):
    def __init__(self, range, wheels, color):
        super().__init__(range)
        self.wheels = wheels
        self.color = color

***We can inspect and use these classes***

In [73]:
mv = MotorVehicle(100)
print(mv)  # Vehicle(range=100, tank=100)
mv.travel(50)  # VOOOM! Traveled 50 kilometers.
mv.travel(30)  # VOOOM! Traveled 30 kilometers.
mv.travel(20)  # Not enough in the tank. Only traveled 20 kilometers.
print(mv)  # Vehicle(range=100, tank=0)
mv.refuel()  # Refueling...
print(mv)  # Vehicle(range=100, tank=100)
c = Car(500, 4, 'red')
print(c.range)  # 500
print(c.tank)  # 500
print(c.wheels)  # 4
print(c.color)  # 'red'
print(c.__dict__)  # {'range': 500, 'tank': 500, 'wheels': 4, 'color': 'red'}
c.travel(50)  # VOOOM! Traveled 50 kilometers.
c.travel(100)  # VOOOM! Traveled 100 kilometers.
c.refuel()  # Refueling...
print(c)  # Vehicle(range=500, tank=500)

Vehicle(range=100, tank=100)
VOOOM! Traveled 50 kilometers.
VOOOM! Traveled 30 kilometers.
VOOOM! Traveled 20 kilometers.
Vehicle(range=100, tank=0)
Refueling...
Vehicle(range=100, tank=100)
500
500
4
red
{'range': 500, 'tank': 500, 'wheels': 4, 'color': 'red'}
VOOOM! Traveled 50 kilometers.
VOOOM! Traveled 100 kilometers.
Refueling...
Vehicle(range=500, tank=500)


## `Handling Errors in Python`


When Python provides errors, we can write code that responds appropriate to them with exceptional control flow.

The try and except blocks are fundamental to handling errors. The optional else and finally blocks are more supplemental, and more rarely seen.

```python
try:
    dangerous_code()
except SomeError:
    handle_the_error()
else: 
    handle_no_error()
finally:
    do_no_matter_what() 
```

The first, and most important step, is that we create an `InvalidPasswordError` that is a subclass of ``ValueError``, so that it's part of the `Exception` hierarchy. With that defined, the `validate_password` function can raise an `InvalidPasswordError` with a helpful message if the supplied password is `invalid`. Finally, the main function will use a `try`/`except`/`else`/`finally` block to 
* 1- attempt to validate a password, 
* 2- respond to any InvalidPasswordError raised in the try block, 
* 3- create an account if there was no InvalidPasswordError, and finally print a generic cleanup message.

In [76]:
class InvalidPasswordError(ValueError):
    pass


INVALID_PASSWORDS = (
    'password',
    'abc123',
    '123abc',
)


def validate_password(username, password):
    if password == username:
        raise InvalidPasswordError("Password cannot be the same as your username.")
    if password in INVALID_PASSWORDS:
        raise InvalidPasswordError("Password cannot one of the most common passwords.")


def create_account(username, password):
    return (username, password)


def main(username, password):
    try:
        validate_password(username, password)
    except InvalidPasswordError as err:
        print(err)
    else:
        account = create_account(username, password)
    finally:
        print("Validated password against username and collection")

In [77]:
main("holaa", "pasps ")

Validated password against username and collection


## C3 Linearization
---
When thinking with attribute resolution, we observed that attribute resolution on class objects proceeds up a linearization of the class object's superclasses. In the case of multiple inheritance, how does a potentially interconnected set of superclasses get linearized? Python uses an algorithm called C3 linearization. The details are complex, but the end result is that Python can linearize the inheritance hierarchy of a class object. An example from that same page:

In [78]:
class A: pass
class B: pass
class C: pass
class D: pass
class E: pass
class K1(A, B, C): pass
class K2(D, B, E): pass
class K3(D, A): pass
class Z(K1, K2, K3): pass

# Print the C3 linearization of Z's superclasses, accessible by the `type.mro` function, which stands for Method Resolution Order.
print(Z.mro())  # [Z, K1, K2, K3, D, A, B, C, E, object]


[<class '__main__.Z'>, <class '__main__.K1'>, <class '__main__.K2'>, <class '__main__.K3'>, <class '__main__.D'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.E'>, <class 'object'>]


In [80]:
InvalidPasswordError.mro()

[__main__.InvalidPasswordError, ValueError, Exception, BaseException, object]

## Slots
We saw that arbitrary attributes can be assigned onto class objects. A class object can define a `__slots__` attribute to a list of attribute names to avoid this behavior. If `__slots__` is defined, its elements are the only acceptable attribute names, and the class object has no `__dict__` attribute. This results in slightly smaller class objects and slightly faster attribute resolution, but removes flexibility.

In [87]:
class NotSlottedExample:
    def __init__(self, name, uid):
        self.name = name
        self.uid = uid


class SlottedExample:
    __slots__ = ['name', 'uid', 'hello']
    def __init__(self, name, uid):
        self.name = name
        self.uid = uid
        self.hello = 244


a = NotSlottedExample('Seb', 1)
b = SlottedExample('Seb', 1)

print(a.name, a.uid)  # ('Seb', 1)
print(b.name, b.uid)  # ('Seb', 1)

print(a.__dict__)  # {'name': 'Seb', 'uid': 1}
print(b.__dict__)
# AttributeError: 'SlottedExample' object has no attribute '__dict__'

a.founder = True  # Okay, no errors here!
b.founder = True
# AttributeError: 'SlottedExample' object has no attribute 'founder'

Seb 1
Seb 1
{'name': 'Seb', 'uid': 1}


AttributeError: 'SlottedExample' object has no attribute '__dict__'